## MTH6412B - Projet Phase 1
#### Victor Darleguy - Nathan Allaire

### Question 1)

In [1]:
include("node.jl")
include("graph.jl")
include("read_stsp.jl")

plot_graph

### Question 2)

Création du type `Edge` :

In [2]:
"""
Type abstrait dont d'autres types d'arêtes dériveront.
"""
abstract type AbstractEdge{T, U} end

"""
Type représentant les arêtes d'un graphe.

Exemple:

    node1 = Node("Joe", 3.14)
    node2 = Node("Steve", exp(1))
    edge = Edge(node1, node2)       # Arête non pondérée
    edge = Edge(node1, node2, 5.6)  # Arête pondérée avec un poids de 5.6
"""
mutable struct Edge{T, U} <: AbstractEdge{T, U}
  start_node::Node{T}
  end_node::Node{T}
  weight::U
end

# Constructeur pour les arêtes non pondérées
Edge(s::Node{T}, e::Node{T}) where T = Edge(s, e, nothing)

# on présume que toutes les arêtes dérivant d'AbstractEdge
# posséderont des champs `start_node`, `end_node` et `weight`.

"""Renvoie le noeud de départ de l'arête."""
start_node(edge::AbstractEdge) = edge.start_node

"""Renvoie le noeud d'arrivée de l'arête."""
end_node(edge::AbstractEdge) = edge.end_node

"""Renvoie le poids de l'arête."""
weight(edge::AbstractEdge) = edge.weight

"""Affiche une arête."""
function show(edge::AbstractEdge)
    if edge.weight === nothing
        println("Edge from ", name(start_node(edge)), " to ", name(end_node(edge)))
    else
        println("Edge from ", name(start_node(edge)), " to ", name(end_node(edge)), ", weight: ", weight(edge))
    end
end  

Base.show

Exemple d'utilisation du type `Edge`:

In [3]:
edge_test = Edge(Node("a", 1.0), Node("b", 2.4))

Edge{Float64, Nothing}(Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), nothing)

In [4]:
show(edge_test)

Edge from a to b


### Question 3)

Extension du type `Graph` pour contenir les arêtes du graphe

In [55]:
# Définir un nouveau type `ExtendedGraph`
mutable struct ExtendedGraph{T} <: AbstractGraph{T}
  name::String
  nodes::Vector{Node{T}}
  edges::Vector{Edge{T}}
end

# Constructeur pour ExtendedGraph
ExtendedGraph(name::String, nodes::Vector{Node{T}}) where T = ExtendedGraph(name, nodes, Edge{T}[])

# Fonction pour ajouter un noeud à un AbstractGraph : 
"""Ajoute un noeud à un ExtendedGraph"""
function add_node!(graph::AbstractGraph{T}, node::Node{T}) where T
  push!(graph.nodes, node)
  graph
end

# Fonction pour ajouter une arête à ExtendedGraph
"""Ajoute une arête à un ExtendedGraph"""
function add_edge!(graph::ExtendedGraph{T}, start_node::Node{T}, end_node::Node{T}, weight = nothing) where {T}
  # Vérifier si les nœuds existent déjà dans le graphe
  if start_node ∉ graph.nodes || end_node ∉ graph.nodes
      throw(ArgumentError("One or both nodes are not part of the graph"))
  end

  # Créer la nouvelle arête
  new_edge = Edge(start_node, end_node, weight)
  
  # Vérifier si l'arête existe déjà
  for edge in graph.edges
      if (edge.start_node == start_node && edge.end_node == end_node) || 
         (edge.start_node == end_node && edge.end_node == start_node)
         return  # simplement retourner si l'arête existe déjà, évitant une erreur

      end
  end

  # Ajouter la nouvelle arête au graphe
  push!(graph.edges, new_edge)
end


add_edge! (generic function with 2 methods)

In [6]:
ext_graph = ExtendedGraph("graph_test", [Node("a", 1.0), Node("b", 2.4), Node("c", 3.0)])

ExtendedGraph{Float64}("graph_test", Node{Float64}[Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), Node{Float64}("c", 3.0)], Edge{Float64}[])

In [7]:
add_edge!(ext_graph, ext_graph.nodes[1], ext_graph.nodes[2])

1-element Vector{Edge{Float64}}:
 Edge{Float64, Nothing}(Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), nothing)

### Question 4)

Implémentation de `show` pour un objet de type `ExtendedGraph`

In [58]:
# Affichage de ExtendedGraph
"""Extension de la fonction `show`pour un ExtendedGraph"""
function show(graph::ExtendedGraph{T}) where {T}
  println("Graph ", name(graph), " has ", nb_nodes(graph), " nodes and ", length(graph.edges), " edges.")
  println("Nodes:")
  for node in nodes(graph)
      show(node)
  end
  println("Edges:")
  for edge in graph.edges
      show(edge)
  end
end

Base.show

In [9]:
show(ext_graph)

Graph graph_test has 3 nodes and 1 edges.
Nodes:
Node a, data: 1.0
Node b, data: 2.4
Node c, data: 3.0
Edges:
Edge from a to b


### Question 5)

In [10]:
header = read_header("instances/stsp/bayg29.tsp")
read_edges(header, "instances/stsp/bayg29.tsp")

406-element Vector{Any}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 6)
 (1, 7)
 (1, 8)
 (1, 9)
 (1, 10)
 (1, 11)
 ⋮
 (25, 27)
 (25, 28)
 (25, 29)
 (26, 27)
 (26, 28)
 (26, 29)
 (27, 28)
 (27, 29)
 (28, 29)

Modification de la méthode `read_edges()` :

In [11]:
"""Analyse un fichier .tsp et renvoie l'ensemble des arêtes sous la forme d'un tableau."""
function read_edges(header::Dict{String, String}, filename::String)

  edges = []
  edge_weight_format = header["EDGE_WEIGHT_FORMAT"]
  known_edge_weight_formats = ["FULL_MATRIX", "UPPER_ROW", "LOWER_ROW",
  "UPPER_DIAG_ROW", "LOWER_DIAG_ROW", "UPPER_COL", "LOWER_COL",
  "UPPER_DIAG_COL", "LOWER_DIAG_COL"]

  if !(edge_weight_format in known_edge_weight_formats)
      @warn "unknown edge weight format" edge_weight_format
      return edges, edge_weights
  end

  file = open(filename, "r")
  dim = parse(Int, header["DIMENSION"])
  edge_weight_section = false
  k = 0
  n_edges = 0
  i = 0
  n_to_read = n_nodes_to_read(edge_weight_format, k, dim)
  flag = false

  for line in eachline(file)
      line = strip(line)
      if !flag
          if occursin(r"^EDGE_WEIGHT_SECTION", line)
              edge_weight_section = true
              continue
          end

          if edge_weight_section
              data = split(line)
              n_data = length(data)
              start = 0
              while n_data > 0
                  n_on_this_line = min(n_to_read, n_data)

                  for j = start : start + n_on_this_line - 1
                    # Les lignes suivantes ont été modifiées pour tenir compte du poids des arêtes
                    n_edges = n_edges + 1
                    edge_weight_value = parse(Float64, data[start+j+1])
                    if edge_weight_format in ["UPPER_ROW", "LOWER_COL"]
                      edge = (k+1, i+k+2, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_DIAG_ROW", "LOWER_DIAG_COL"]
                      edge = (k+1, i+k+1, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_COL", "LOWER_ROW"]
                      edge = (i+k+2, k+1, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_DIAG_COL", "LOWER_DIAG_ROW"]
                      edge = (i+1, k+1, edge_weight_value)
                    elseif edge_weight_format == "FULL_MATRIX"
                      edge = (k+1, i+1, edge_weight_value)
                    else
                      warn("Unknown format - function read_edges")
                    end
                    push!(edges, edge)
                    i += 1
                  end

                  n_to_read -= n_on_this_line
                  n_data -= n_on_this_line

                  if n_to_read <= 0
                      start += n_on_this_line
                      k += 1
                      i = 0
                      n_to_read = n_nodes_to_read(edge_weight_format, k, dim)
                  end

                  if k >= dim
                      n_data = 0
                      flag = true
                  end
              end
          end
      end
  end
  close(file)
  return edges
end


read_edges

In [12]:
r = read_edges(header, "instances/stsp/bayg29.tsp")

406-element Vector{Any}:
 (1, 2, 97.0)
 (1, 3, 205.0)
 (1, 4, 139.0)
 (1, 5, 86.0)
 (1, 6, 60.0)
 (1, 7, 220.0)
 (1, 8, 65.0)
 (1, 9, 111.0)
 (1, 10, 115.0)
 (1, 11, 227.0)
 ⋮
 (25, 27, 120.0)
 (25, 28, 205.0)
 (25, 29, 270.0)
 (26, 27, 213.0)
 (26, 28, 145.0)
 (26, 29, 36.0)
 (27, 28, 94.0)
 (27, 29, 217.0)
 (28, 29, 162.0)

### Question 6)

L'instance `bays29.tsp` est une matrice pleine symétrique dont les poids sont donnés au format `EXPLICIT`

In [13]:
header = read_header("instances/stsp/bays29.tsp")
nodes_graph = read_nodes(header, "instances/stsp/bays29.tsp")
edges_graph = read_edges(header, "instances/stsp/bays29.tsp")

841-element Vector{Any}:
 (1, 1, 0.0)
 (1, 2, 107.0)
 (1, 3, 241.0)
 (1, 4, 190.0)
 (1, 5, 124.0)
 (1, 6, 80.0)
 (1, 7, 316.0)
 (1, 8, 76.0)
 (1, 9, 152.0)
 (1, 10, 157.0)
 ⋮
 (29, 21, 108.0)
 (29, 22, 332.0)
 (29, 23, 342.0)
 (29, 24, 218.0)
 (29, 25, 350.0)
 (29, 26, 39.0)
 (29, 27, 263.0)
 (29, 28, 199.0)
 (29, 29, 0.0)

Nous avons créé un vecteur contenant les arêtes et les poids des arêtes. Construisons à présent l'objet `Graph` correspondant :  

In [56]:
graph_bays29 = ExtendedGraph("bays29", [Node("1", nodes_graph[1])])
for i=2:length(nodes_graph)
  node = Node(string(i), nodes_graph[i])
  add_node!(graph_bays29, node)
end

for i=1:length(edges_graph)
  s_node = graph_bays29.nodes[edges_graph[i][1]]
  e_node = graph_bays29.nodes[edges_graph[i][2]]
  w_edge = edges_graph[i][3]
  add_edge!(graph_bays29, s_node, e_node, w_edge)
end


In [57]:
show(graph_bays29)

Graph bays29 has 29 nodes and 435 edges.
Nodes:
Node 1, data: [1150.0, 1760.0]
Node 2, data: [630.0, 1660.0]
Node 3, data: [40.0, 2090.0]
Node 4, data: [750.0, 1100.0]
Node 5, data: [750.0, 2030.0]
Node 6, data: [1030.0, 2070.0]
Node 7, data: [1650.0, 650.0]
Node 8, data: [1490.0, 1630.0]
Node 9, data: [790.0, 2260.0]
Node 10, data: [710.0, 1310.0]
Node 11, data: [840.0, 550.0]
Node 12, data: [1170.0, 2300.0]
Node 13, data: [970.0, 1340.0]
Node 14, data: [510.0, 700.0]
Node 15, data: [750.0, 900.0]
Node 16, data: [1280.0, 1200.0]
Node 17, data: [230.0, 590.0]
Node 18, data: [460.0, 860.0]
Node 19, data: [1040.0, 950.0]
Node 20, data: [590.0, 1390.0]
Node 21, data: [830.0, 1770.0]
Node 22, data: [490.0, 500.0]
Node 23, data: [1840.0, 1240.0]
Node 24, data: [1260.0, 1500.0]
Node 25, data: [1280.0, 790.0]
Node 26, data: [490.0, 2130.0]
Node 27, data: [1460.0, 1420.0]
Node 28, data: [1260.0, 1910.0]
Node 29, data: [360.0, 1980.0]
Edges:
Edge from 1 to 1, weight: 0.0
Edge from 1 to 2, weigh

 to 14, weight: 186.0
Edge from 13 to 15, weight: 117.0
Edge from 13 to 16, weight: 75.0
Edge from 13 to 17, weight: 231.0
Edge from 13 to 18, weight: 165.0
Edge from 13 to 19, weight: 81.0
Edge from 13 to 20, weight: 85.0
Edge from 13 to 21, weight: 92.0
Edge from 13 to 22, weight: 230.0
Edge from 13 to 23, weight: 184.0
Edge from 13 to 24, weight: 74.0
Edge from 13 to 25, weight: 150.0
Edge from 13 to 26, weight: 208.0
Edge from 13 to 27, weight: 104.0
Edge from 13 to 28, weight: 158.0
Edge from 13 to 29, weight: 206.0
Edge from 14 to 14, weight: 0.0
Edge from 14 to 15, weight: 69.0
Edge from 14 to 16, weight: 191.0
Edge from 14 to 17, weight: 59.0
Edge from 14 to 18, weight: 35.0
Edge from 14 to 19, weight: 125.0
Edge from 14 to 20, weight: 167.0
Edge from 14 to 21, weight: 255.0
Edge from 14 to 22, weight: 44.0
Edge from 14 to 23, weight: 309.0
Edge from 14 to 24, weight: 245.0
Edge from 14 to 25, weight: 169.0
Edge from 14 to 26, weight: 327.0
Edge from 14 to 27, weight: 246.0
Edg

### Question 7)

c.f fichier `main.jl` à l'url : https://github.com/nathanemac/Projet/tree/phase1/Phase%201